In [ ]:
import numpy as np

In [ ]:
def MSD(target_patch, exemplar_patch, M_bar):
    msd=np.sum((M_bar*target_patch - M_bar*exemplar_patch)**2)/np.sum(M_bar)
    return msd

def SMD (target_patch, exemplar_patch, binary_mask, M_bar):
    target_existing_avg= np.sum(M_bar * target_patch)/np.sum(M_bar)
    exemplar_fill_avg = np.sum(binary_mask* exemplar_patch)/np.sum(binary_mask)
    smd= (target_existing_avg-exemplar_fill_avg)**2
    return smd 

def findBestsubPatch(target_patch, exemplar_patches, binary_mask, M_bar):
    best_patch=None
    min_combined_error= float('inf')
    for current_patch,other_patch in zip(target_patch,exemplar_patches):
        current_error=MSD(current_patch,other_patch,M_bar)+ SMD(current_patch,other_patch,binary_mask,M_bar)
        if current_error < min_combined_error:
            best_patch = other_patch
    return best_patch

def findBestFullPatch(target_patches, exemplar_patches,binary_mask_patches, M_bar_patches):
    cp_target_patches = np.copy(target_patches)
    for i,target_patch in enumerate(target_patches):
        best_sub_patch=findBestsubPatch(target_patch,exemplar_patches,binary_mask_patches[i],M_bar_patches[i])
        cp_target_patches[i]=best_sub_patch[binary_mask_patches[i]*best_sub_patch]
    return cp_target_patches


In [ ]:
def main(img,binary_mask):
    patch_size = 8
    img_in_patches= [img[i:i+patch_size,j:j+patch_size] for i in range(0,img.shape[0],patch_size) for j in range(0,img.shape[1],patch_size) ]
    full_binary_mask_patches = [binary_mask[i:i+patch_size,j:j+patch_size] for i in range(0,img.shape[0],patch_size) for j in range(0,img.shape[1],patch_size) ]
    #if the patch contains a single 1 then it's subset of the target patch so mark it with 1 in the bit map
    bit_map=[ 1 if np.isin(1,full_binary_mask_patches[i]) else 0 for i in range(len(full_binary_mask_patches))] 
    binary_mask_patches= []
    target_patches=[]
    exemplar_patches=[]
    for i in range(len(bit_map)):
        if bit_map[i] == 1:
            binary_mask_patches.append(full_binary_mask_patches[i])
            target_patches.append(img_in_patches[i])
        else:
            exemplar_patches.append(img_in_patches[i])
    M_bar_patches=np.where((binary_mask_patches==0)|(binary_mask_patches==1), binary_mask_patches^1, binary_mask_patches)
    #TO-DO: call implemented functions